---
format:
    html:
        embed-resources: true
---

# Resume and cover letter creator 

Using a combination of an LLM API wrapper and Quarto, write a pipeline that will create a resume PDF (or HTML), and a cover letter, for each of the given job descriptions. 

In [5]:
import pandas as pd
import numpy as np

## API

In [1]:
import google.generativeai as genai

In [2]:
import json
with open('C:/Users/admin/.google-api.json') as f:
    keys = json.load(f)

In [3]:
API_KEY = keys['googleapi']

In [4]:
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-1.5-flash")

### Define the Input Data

In [6]:
df = pd.read_csv("df_cleaned.csv")

## Use LLM for Content Generation

### Prompt Engineering for Resume Content

In [11]:
import json
from pprint import pprint

# Initialize the generative model
model = genai.GenerativeModel("gemini-1.5-flash")

# Define placeholder profile for Jane Doe
jane_doe_profile = {
    "Name": "Jane Doe",
    "Contact Information": {
        "Email": "jane.doe@example.com",
        "Phone": "(123) 456-7890",
        "Location": "New York, NY",
        "LinkedIn": "linkedin.com/in/jane-doe",
        "Portfolio": "janedoe-portfolio.com"
    },
    "Professional Summary": "Data Analyst with 3+ years of experience in data analysis, visualization, and machine learning. Skilled in SQL, Python, and data visualization tools, with a strong focus on delivering actionable insights.",
    "Education": {
        "Degree": "Bachelor of Science in Computer Science",
        "University": "University of Example",
        "Location": "New York, NY",
        "Graduation Date": "May 2018",
        "Relevant Courses": ["Data Structures", "Algorithms", "Machine Learning", "Database Systems"]
    },
    "Work Experience": [
        {
            "Title": "Data Analyst",
            "Company": "Example Corp",
            "Location": "New York, NY",
            "Date": "July 2018 – Present",
            "Responsibilities": [
                "Conducted data analysis and visualization to support business decision-making.",
                "Developed SQL queries to retrieve data for analysis and reporting.",
                "Collaborated with cross-functional teams to deliver insights on customer behavior, market trends, and operational efficiency."
            ],
            "Tools Used": ["SQL", "Python", "Tableau"]
        },
        {
            "Title": "Intern - Data Science",
            "Company": "Example Inc.",
            "Location": "New York, NY",
            "Date": "May 2017 – August 2017",
            "Responsibilities": [
                "Assisted in data cleaning, feature engineering, and predictive modeling.",
                "Presented insights to senior management, improving model accuracy by 15%."
            ],
            "Tools Used": ["Python", "R", "Excel"]
        }
    ],
    "Skills": {
        "Technical": ["Python", "SQL", "R", "Tableau", "Machine Learning", "Data Analysis", "Data Visualization"],
        "Soft": ["Communication", "Problem Solving", "Team Collaboration", "Adaptability", "Critical Thinking"]
    },
    "Certifications": [
        "Google Data Analytics Professional Certificate",
        "Microsoft Certified: Azure Data Scientist Associate"
    ],
    "Projects": [
        {
            "Name": "Customer Segmentation Analysis",
            "Description": "Created customer segments using clustering algorithms for targeted marketing campaigns."
        },
        {
            "Name": "Sales Forecasting Model",
            "Description": "Developed a forecasting model to predict sales trends, reducing inventory costs by 10%."
        }
    ],
    "Languages": ["English (Fluent)", "Spanish (Conversational)"],
    "Interests": ["Data Ethics", "AI for Social Good", "Tech Blogging"]
}

# Function to generate resume content based on prompts
def generate_resume_content(profile, content_type):
    """
    Generate specific resume content based on Jane Doe's profile using Gemini API.
    
    Parameters:
    profile (dict): The Jane Doe profile data.
    content_type (str): The type of content to generate ("summary", "skills", "experience", or "education").
    
    Returns:
    str: Generated content as text.
    """
    # Define prompts for different resume sections
    prompts = {
        "summary": f"Create a professional summary based on Jane Doe's profile: {profile['Professional Summary']}.",
        "skills": f"List the relevant skills for Jane Doe based on her technical and soft skills: {profile['Skills']}.",
        "experience": f"Describe Jane Doe's relevant experience for a data analyst role, focusing on the experience sections: {profile['Work Experience']}.",
        "education": f"Provide a brief educational background for Jane Doe, focusing on her degree and relevant courses: {profile['Education']}."
    }
    
    # Generate content using the specified prompt
    response = model.generate_content(prompts[content_type])
    return response.text.strip()

# Generate and store the resume content based on Jane Doe's profile
resume_content = {
    "Summary": generate_resume_content(jane_doe_profile, "summary"),
    "Skills": generate_resume_content(jane_doe_profile, "skills"),
    "Experience": generate_resume_content(jane_doe_profile, "experience"),
    "Education": generate_resume_content(jane_doe_profile, "education")
}

# Save generated resume content to JSON file for later use
with open("jane_doe_generated_resume.json", "w") as f:
    json.dump(resume_content, f, indent=4)

# Display the generated resume content for verification
pprint(resume_content)

{'Education': 'Jane Doe holds a Bachelor of Science in Computer Science from '
              'the University of Example in New York, NY, graduating in May '
              '2018. Her coursework focused on core computer science '
              'principles, including Data Structures, Algorithms, Machine '
              'Learning, and Database Systems. This foundation provides her '
              'with a strong understanding of computational theory and '
              'practical skills for working with data and developing software '
              'applications.',
 'Experience': "## Jane Doe's Relevant Experience for a Data Analyst Role\n"
               '\n'
               'Jane Doe boasts a strong background in data analysis, '
               'demonstrated through her experience at both **Example Corp** '
               'and **Example Inc**. \n'
               '\n'
               '**At Example Corp, as a Data Analyst (July 2018 – Present):**\n'
               '\n'
               '* Jane h

### Prompt Engineering for Cover Letter

In [15]:
# Initialize the generative model
model = genai.GenerativeModel("gemini-1.5-flash")

# Load the first 10 rows from CSV
df = pd.read_csv("df_cleaned.csv").head(10)

# Define placeholder profile for Jane Doe
jane_doe_profile = {
    "Name": "Jane Doe",
    "Contact Information": {
        "Email": "jane.doe@example.com",
        "Phone": "(123) 456-7890",
        "Location": "New York, NY"
    },
    "Work Experience": [
        {
            "Title": "Data Analyst",
            "Company": "Example Corp",
            "Responsibilities": [
                "Conducted data analysis and visualization to support business decision-making.",
                "Developed SQL queries to retrieve data for analysis and reporting.",
                "Collaborated with cross-functional teams to deliver insights on customer behavior, market trends, and operational efficiency."
            ]
        }
    ],
    "Skills": {
        "Technical": ["Python", "SQL", "R", "Tableau", "Machine Learning", "Data Analysis"],
        "Soft": ["Communication", "Problem Solving", "Team Collaboration", "Adaptability", "Critical Thinking"]
    }
}

# Function to construct a cover letter prompt based on job description
def construct_cover_letter_prompt(row, profile):
    """
    Construct a cover letter prompt using job description details and Jane Doe's profile.
    
    Parameters:
    row (pd.Series): A row from the dataframe representing a job description.
    profile (dict): The Jane Doe profile data.
    
    Returns:
    str: A formatted prompt for generating a cover letter.
    """
    prompt = (
        f"Write a cover letter for {profile['Name']} applying for the role of {row['title']} at {row['company_name']}.\n"
        f"Include an introduction mentioning her interest in the position, her relevant experience as described below, "
        f"and her skills that match the job requirements.\n"
        f"{profile['Name']}'s contact details are:\n"
        f"Email: {profile['Contact Information']['Email']}, Phone: {profile['Contact Information']['Phone']}, Location: {profile['Contact Information']['Location']}.\n"
        f"The job was advertised on {row['via']}.\n"
        f"Relevant experience:\n"
        f"{profile['Work Experience'][0]['Responsibilities']}.\n"
        f"Her key skills are: {', '.join(profile['Skills']['Technical'] + profile['Skills']['Soft'])}.\n"
        f"Conclude with a polite closing statement expressing appreciation and a desire for an interview opportunity."
    )
    return prompt

# Dictionary to store generated cover letters
cover_letters = {}

# Generate cover letters for the first 10 job descriptions
for idx, row in df.iterrows():
    # Construct prompt for each job
    prompt = construct_cover_letter_prompt(row, jane_doe_profile)
    
    # Generate cover letter content using the Gemini model
    response = model.generate_content(prompt)
    cover_letters[f"Job_{idx+1}"] = response.text.strip()

# Save generated cover letters to a JSON file for later use
with open("jane_doe_cover_letters.json", "w") as f:
    json.dump(cover_letters, f, indent=4)

# Print generated cover letters for verification
pprint(cover_letters)

{'Job_1': '[Your Name]\n'
          '[Your Address] | [Your Phone Number] | [Your Email]\n'
          '\n'
          '[Date]\n'
          '\n'
          'Hiring Manager\n'
          'Photon\n'
          '[Company Address]\n'
          '\n'
          '**Subject: Application for AI Solution Architect - Azure**\n'
          '\n'
          'Dear Hiring Manager,\n'
          '\n'
          'I am writing to express my enthusiastic interest in the AI Solution '
          'Architect - Azure position at Photon, as advertised on Indeed. With '
          'my proven experience in data analysis, data visualization, and AI '
          'technologies, I am confident in my ability to contribute '
          'significantly to your team.\n'
          '\n'
          'Throughout my career, I have consistently demonstrated my expertise '
          'in leveraging data to drive impactful business decisions. My '
          'experience includes:\n'
          '\n'
          '* **Conducted data analysis and visual

## Organize and Structure Content in Quarto

In [19]:
import json

# Load generated content from JSON files
with open("jane_doe_generated_resume.json") as resume_file:
    resume_data = json.load(resume_file)

with open("jane_doe_cover_letters.json") as cover_letter_file:
    cover_letter_data = json.load(cover_letter_file)

# Define a function to populate the Quarto template with data
def populate_template(template_path, output_path, data):
    with open(template_path, "r") as file:
        template = file.read()

    # Ensure Skills is structured correctly
    if isinstance(data.get("Skills"), dict):
        skills_technical = ", ".join(data["Skills"].get("Technical", []))
        skills_soft = ", ".join(data["Skills"].get("Soft", []))
    else:
        skills_technical = ""
        skills_soft = ""

    # Replace placeholders in the template with actual data
    filled_content = template.replace("{{summary}}", data.get("Summary", ""))
    filled_content = filled_content.replace("{{skills_technical}}", skills_technical)
    filled_content = filled_content.replace("{{skills_soft}}", skills_soft)
    filled_content = filled_content.replace("{{experience}}", data.get("Experience", ""))
    filled_content = filled_content.replace("{{education}}", data.get("Education", ""))
    filled_content = filled_content.replace("{{cover_letter}}", cover_letter_data.get("Job_1", ""))  # Example using the first job's cover letter

    # Write the filled content to the output file
    with open(output_path, "w") as output_file:
        output_file.write(filled_content)

# Populate the template
populate_template("jane_doe_resume.qmd", "filled_jane_doe_resume.qmd", resume_data)


In [21]:
import subprocess

# Run the Quarto render command to generate a PDF
subprocess.run(["quarto", "render", "filled_jane_doe_resume.qmd", "--to", "pdf"])

CompletedProcess(args=['quarto', 'render', 'filled_jane_doe_resume.qmd', '--to', 'pdf'], returncode=1)